Indexing dataset
================

In [1]:
from pathlib import Path
import sys
debug_local = True#False
local = (Path("..") / "indexpaper").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  print("extending pathes with local indexpaper")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local indexpaper
['..', '/home/antonkulaga/sources/indexpaper/notebooks', '/home/antonkulaga/sources/indexpaper', '/home/antonkulaga/micromamba/envs/indexpaper/lib/python310.zip', '/home/antonkulaga/micromamba/envs/indexpaper/lib/python3.10', '/home/antonkulaga/micromamba/envs/indexpaper/lib/python3.10/lib-dynload', '', '/home/antonkulaga/.local/lib/python3.10/site-packages', '/home/antonkulaga/micromamba/envs/indexpaper/lib/python3.10/site-packages']


In [2]:
import polars as pl
from indexpaper import index

from pycomfort.files import *
from typing import List
from pathlib import Path
from polars import Config
Config.set_fmt_str_lengths(5000)

/home/antonkulaga/.local/lib/python3.10/site-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Callable[[list], list] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


polars.config.Config

In [9]:
from pycomfort.config import load_environment_keys
from indexpaper.paperset import Paperset
from indexpaper.resolvers import *

chunk_size = 512
embeddings, splitter = resolve_embedding_splitter(EmbeddingType.HuggingFace, EmbeddingModels.biolinkbert.value, chunk_size=chunk_size)
from indexpaper.indexing import init_qdrant
load_environment_keys(usecwd=True)
api_key = os.getenv("QDRANT_KEY")
db = init_qdrant("biolinkbert_large_512_tacutu_papers", 
                 path_or_url="https://5bea7502-97d4-4876-98af-0cdf8af4bd18.us-east-1-0.aws.cloud.qdrant.io:6333/", 
                 embeddings=embeddings, always_recreate=False,
                 api_key=api_key
                 )

No sentence-transformers model found with name /home/antonkulaga/.cache/torch/sentence_transformers/michiyasunaga_BioLinkBERT-large. Creating a new one with MEAN pooling.
2023-08-23 00:40:06.328 | INFO     | indexpaper.indexing:init_qdrant:78 - initializing quadrant database at https://5bea7502-97d4-4876-98af-0cdf8af4bd18.us-east-1-0.aws.cloud.qdrant.io:6333/


/home/antonkulaga/sources/indexpaper/.env
environment found at /home/antonkulaga/sources/indexpaper/.env


In [10]:
col = db.client.get_collection("biolinkbert_large_512_tacutu_papers")
col

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=39, indexed_vectors_count=0, points_count=39, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={'pubmed': PayloadIndexInfo(data_type=<PayloadSchemaType.TEXT: 'text'>, par

In [24]:
from qdrant_client.http.models import *
from typing import Union
#TODO: fix it does not work!
db.client.search("biolinkbert_large_512_tacutu_papers", None,
    Filter(
    must=[
        FieldCondition(
            key='doi',
            match=MatchValue(value = "10.1186/s13059-020-01990-9")
        )
    ]
))

ValidationError: 1 validation error for SearchRequest
vector
  none is not an allowed value (type=type_error.none.not_allowed)

# Paperset #

In [ ]:
moskalev = Paperset("longevity-genie/moskalev_papers", splitter=splitter)
moskalev.lazy_frame.collect().head(1)

In [4]:
col = db.client.get_collections("longevity-genie/moskalev_papers")
col

NameError: name 'db' is not defined

Indexing
========
To avoid memory overflow we index everything by slices

In [8]:
moskalev.index_by_slices(10, db, 0)

2023-08-06 14:33:01.672 | INFO     | indexpaper.paperset:index_fun:168 - indexed 0 + 10 * 792 documents with last document having 10.3389/fphar.2016.00505
2023-08-06 14:35:12.323 | INFO     | indexpaper.paperset:index_fun:168 - indexed 0 + 10 * 508 documents with last document having 10.3389/fgene.2015.00353
2023-08-06 14:38:44.692 | INFO     | indexpaper.paperset:index_fun:168 - indexed 0 + 10 * 858 documents with last document having 10.18632/oncotarget.24461
2023-08-06 14:41:27.001 | INFO     | indexpaper.paperset:index_fun:168 - indexed 0 + 10 * 579 documents with last document having 10.3389/fgene.2019.00303
2023-08-06 14:43:45.845 | INFO     | indexpaper.paperset:index_fun:168 - indexed 0 + 10 * 561 documents with last document having 10.3389/fgene.2018.00095
2023-08-06 14:47:41.329 | INFO     | indexpaper.paperset:index_fun:168 - indexed 0 + 10 * 803 documents with last document having 10.3389/fmed.2020.00371
2023-08-06 14:50:21.931 | INFO     | indexpaper.paperset:index_fun:168

In [9]:
# Testing index


In [17]:
db.similarity_search("IKK-β activation ")[0].page_content

'proteins: p65 (RelA), RelB, c-Rel, p105/p50 (NF-κB1), and p100/52 (NF-κB2), which associate with each other to form distinct active NF-κB dimers. In the cytosol, NF-κB dimers in inactive form anchored by IκB are found. There are two ways to activate the NF-kB-induced gene transcription. These are triggered by cytokines TNFα and IL-1 (canonical signaling) or antigen receptors CD40 and BAFF (non-canonical/alternative signaling). There are the activation of IKK complex (IKKα, IKKβ, and IKKγ) and the phosphorylation of IκB proteins, which are in interaction with'